In [1]:
import os
from tqdm import tqdm
import argparse

import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("sample_train.ipynb"))))
from common.parser import yaml_parser
from common.recoder import save_checkpoint
from data.yolo_dataset import *
from model.loss import YOLOv3Loss
from model.model import Darknet4YOLOv3

import torch
import torch.nn
from torch.utils.tensorboard import SummaryWriter

In [2]:
import easydict

args = easydict.EasyDict({
    "config": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolov3.cfg",
    # "weight": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.weights",
    "dataset": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/dataset/yolo_dataset.yml",
    "model": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolo_model.yml",
    "optimizer": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/optimizer/optimizer.yml",
    "weight_save_dir": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/weights"
})


dataset_option = yaml_parser(args.dataset)
model_option = yaml_parser(args.model)
optimizer_option = yaml_parser(args.optimizer)

In [3]:
model = Darknet4YOLOv3(args.config)

In [5]:
# device = torch.device('cpu')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###########################
## BUILD MODEL & LOSS_fn ##
###########################
# model = DarknetParser(args.config, args.weight)
model = Darknet4YOLOv3(args.config).to(device)


In [6]:
model.train()

Darknet4YOLOv3(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): BatchNorm2d(64, eps=1e-05